In [1]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
import os
import matplotlib

matplotlib.use("Agg")
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_score
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score

import numpy as np

import pickle

In [3]:
# dataset = "CHASEDB1"
# dim = "_128" if dataset == "DRIVE" else "_192"
# # dim = ""

# pred_file = "preds_EffnetB7_DenseUnet_2021-02-12 08:20:38.npz"

# main_path = f"/content/gdrive/My Drive/Data/{dataset}"

In [4]:
dataset = "DRIVE"
dim = "_128" if dataset == "DRIVE" else "_192"
# dim = ""

pred_file = "preds_EffnetB7_DenseUnet_2021-02-12 18:02:50.npz"

main_path = f"/content/gdrive/My Drive/Data/{dataset}"

In [5]:
testing_labels = f"{main_path}/patches{dim}/labels/test/"

test_lbl = next(os.walk(testing_labels))[2]

Y_test = np.concatenate([np.load(testing_labels + file_id)["arr_0"] for file_id in test_lbl], axis=0)

if dataset == "CHASEDB1":
    Y_test = Y_test.astype("float64")
else:
    Y_test = Y_test / 255

In [6]:
print(np.max(Y_test))
print(np.min(Y_test))

1.0
0.0


In [7]:
Y_test.shape

(800, 128, 128)

In [8]:
Y_pred = np.load(f"{main_path}/patches{dim}/predictions/{pred_file}")["arr_0"]
Y_pred = np.reshape(Y_pred, (Y_pred.shape[0], Y_pred.shape[1], Y_pred.shape[2]))

In [9]:
Y_pred.shape

(800, 128, 128)

In [10]:
def evaluate(y_true, y_scores):
    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    # Area under the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    AUC_ROC = roc_auc_score(y_true, y_scores)
    print("\nArea under the ROC curve: " + str(AUC_ROC))
    plt.plot(fpr, tpr, '-')
    plt.title('ROC curve', fontsize=14)
    plt.xlabel("FPR (False Positive Rate)", fontsize=15)
    plt.ylabel("TPR (True Positive Rate)", fontsize=15)
    plt.legend(loc="lower right")
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # Precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    precision = np.fliplr([precision])[0]  # so the array is increasing (you won't get negative AUC)
    recall = np.fliplr([recall])[0]  # so the array is increasing (you won't get negative AUC)
    AUC_prec_rec = np.trapz(precision, recall)
    print("\nArea under Precision-Recall curve: " + str(AUC_prec_rec))

    # Confusion matrix
    threshold_confusion = 0.5
    print("\nConfusion matrix:  Costum threshold (for positive) of " + str(threshold_confusion))
    y_pred = np.empty((y_scores.shape[0]))
    for i in range(y_scores.shape[0]):
        if y_scores[i] >= threshold_confusion:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    confusion = confusion_matrix(y_true, y_pred)
    print(confusion)
    accuracy = 0
    if float(np.sum(confusion)) != 0:
        accuracy = float(confusion[0, 0] + confusion[1, 1]) / float(np.sum(confusion))
    print("Global Accuracy: " + str(accuracy))
    specificity = 0
    if float(confusion[0, 0] + confusion[0, 1]) != 0:
        specificity = float(confusion[0, 0]) / float(confusion[0, 0] + confusion[0, 1])
    print("Specificity: " + str(specificity))
    sensitivity = 0
    if float(confusion[1, 1] + confusion[1, 0]) != 0:
        sensitivity = float(confusion[1, 1]) / float(confusion[1, 1] + confusion[1, 0])
    print("Sensitivity: " + str(sensitivity))
    precision = 0
    if float(confusion[1, 1] + confusion[0, 1]) != 0:
        precision = float(confusion[1, 1]) / float(confusion[1, 1] + confusion[0, 1])
    print("Precision: " + str(precision))

    # Jaccard similarity index
    jaccard_index = jaccard_similarity_score(y_true, y_pred, normalize=True)
    print("\nJaccard similarity score: " + str(jaccard_index))

    # F1 score
    F1_score = f1_score(y_true, y_pred, labels=None, average='binary', sample_weight=None)
    print("\nF1 score (F-measure): " + str(F1_score))
    # Save the results
    # file_perf = open(f'./output/{dataset}/performances.txt', 'w')
    # file_perf.write("Area under the ROC curve: " + str(AUC_ROC)
    #                 + "\nArea under Precision-Recall curve: " + str(AUC_prec_rec)
    #                 + "\nJaccard similarity score: " + str(jaccard_index)
    #                 + "\nF1 score (F-measure): " + str(F1_score)
    #                 + "\n\nConfusion matrix:"
    #                 + str(confusion)
    #                 + "\nACCURACY: " + str(accuracy)
    #                 + "\nSENSITIVITY: " + str(sensitivity)
    #                 + "\nSPECIFICITY: " + str(specificity)
    #                 + "\nPRECISION: " + str(precision)
    #                 )
    # file_perf.close()

    plt.show()

In [11]:
Y_test = Y_test.flatten()
Y_pred = Y_pred.flatten()

In [12]:
evaluate(Y_test, Y_pred)

No handles with labels found to put in legend.



Area under the ROC curve: 0.9780922715509381

Area under Precision-Recall curve: 0.8792472797190534

Confusion matrix:  Costum threshold (for positive) of 0.5
[[11237443   395232]
 [  236935  1237590]]
Global Accuracy: 0.9517694854736328
Specificity: 0.9660239798670555
Sensitivity: 0.8393143554704057
Precision: 0.7579454465949136


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  FutureWarning)



Jaccard similarity score: 0.9517694854736328

F1 score (F-measure): 0.7965573204408777
